In [1]:
# start with a category
# go over all pages to collect comics names under this category
# for each name go over all dates and collect them: 
#    need to first open the current page (click on left most and then click on right and collect)

# how could we get all categories for each comic?

In [30]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import os
import pickle

In [24]:
go_comics_url = 'https://www.gocomics.com'

In [15]:
os.environ['https_proxy'] = 'http://proxy.cmu.edu:3128/'

In [25]:
urlopen(go_comics_url)

In [34]:
req = Request('http://www.example.com/')
req.add_header('User-Agent', 'Mozilla/5.0')
resp = urlopen(req)

In [33]:
resp

In [271]:
for category in ['family', 'office', 'gag', 'political', ]:
    if not os.path.exists('./comics/' + category):
        os.makedirs('./comics/' + category)

    print ('----', category, '----')
    category_urls = []
    category_url = go_comics_url + '/comics/' + category
    category_urls.append(category_url)
    
    category_page = urlopen(category_url)
    category_html = category_page.read().decode("utf-8")
    category_soup = BeautifulSoup(category_html, "html.parser")
    category_total_pages = category_soup.select("[class=gc-pagination__item]")
    for page in category_total_pages:
        category_urls.append(go_comics_url + page['href'])
    category_comics = {}
    
    for category_url in list(set(category_urls)):
        print ('--', category_url, '--')
        category_page = urlopen(category_url)
        category_html = category_page.read().decode("utf-8")
        category_soup = BeautifulSoup(category_html, "html.parser")
        current_page_comics = category_soup.select("[class=gc-blended-link\ gc-blended-link--primary]")
        
        
        for comic in current_page_comics:
            comic_name = comic['href'].split('/')[1]
            category_comics[comic_name] = {}
            current_comic_url = ''
            if not os.path.exists('./comics/' + category + '/' + comic_name):
                os.makedirs('./comics/' + category + '/' + comic_name)
            
            comic_url = go_comics_url + '/' + comic_name
            comic_page = urlopen(comic_url)
            comic_html = comic_page.read().decode("utf-8")
            comic_soup = BeautifulSoup(comic_html, "html.parser")
            
            comic_earliest_url = ''
            comic_current_year = 2021
            comic_current_month = 12
            comic_current_day = 31
            
            for date_url_entry in comic_soup.select('[class=gc-blended-link\ gc-blended-link--primary]'):
                date_url = date_url_entry['href']
                tokens = date_url.split('/')

                if len(tokens) == 5 and ('http' not in tokens[0]):
                    tmp_year = tokens[2]
                    try:
                        if int(tmp_year) <= comic_current_year:
                            comic_current_year = int(tmp_year)
                            tmp_month = tokens[3]
                            comic_earliest_url = date_url
                            if int (tmp_month) <= comic_current_month:
                                comic_current_month = int(tmp_month)
                                tmp_day = tokens[4]
                                comic_earliest_url = date_url
                                if int (tmp_day) <= comic_current_day:
                                    comic_current_day = int(tmp_day)
                                    comic_earliest_url = date_url
                    except:
                        print ('comic year extracting error: ', tokens)
            print (comic_name, comic_earliest_url)
            category_comics[comic_name]['earliest_url'] = comic_earliest_url
            current_comic_url = 'https://www.gocomics.com' + earliest_url
            
            while True:
                current_comic_page = urlopen(current_comic_url)
                current_comic_html = current_comic_page.read().decode("utf-8")
                current_comic_soup = BeautifulSoup(current_comic_html, "html.parser")
                
                my_comic = current_comic_soup.select('[itemtype=http\:\/\/schema\.org\/CreativeWork]')[0]
                category_comics[comic_name][current_comic_url] = {}
                category_comics[comic_name][current_comic_url]['image_url'] = my_comic.get_attribute_list('data-image')[0]
                category_comics[comic_name][current_comic_url]['likes'] = int(current_comic_soup.select('[class=js-like]')[0].get_text())
                category_comics[comic_name][current_comic_url]['favorites'] = int(
                    current_comic_soup.select('[class=js-favorite]')[0].get_text())
                category_comics[comic_name][current_comic_url]['comments'] = int(current_comic_soup.select(
                    '[class=gc-comment-toggle]')[0].get_text())
                category_comics[comic_name][current_comic_url]['date'] = my_comic.get_attribute_list(
                    'data-formatted-date')[0].replace(', ', '_').replace(' ', '_')
                
                #download the image, pickle once in a while
                img_data = requests.get(category_comics[comic_name][current_comic_url]['image_url'] ).content
                with open( './comics/' + category + '/' + comic_name + '/'+ 
                          category_comics[comic_name][current_comic_url]['date'] +'.jpg', 'wb') as handler:
                    handler.write(img_data)
                               
            
                if len(current_comic_soup.select(
                '[class=fa\ btn\ btn-outline-secondary\ btn-circle\ fa-caret-right\ sm\ disabled]')) == 1:
                    break
                
                next_comic_date = current_comic_soup.select(
                    '[class=fa\ btn\ btn-outline-secondary\ btn-circle\ fa-caret-right\ sm]')[0]['href']
                current_comic_url = 'https://www.gocomics.com' + next_comic_date
        
        with open('./comics/'+ category + '/data.pickle', 'wb') as handle:
            pickle.dump(category_comics, handle, protocol=pickle.HIGHEST_PROTOCOL)

---- family ----
-- https://www.gocomics.com/comics/family --
adamathome /adamathome/1995/06/20


KeyboardInterrupt: 

In [275]:
category_comics

{'adamathome': {'earliest_url': '/adamathome/1995/06/20',
  'https://www.gocomics.com/adamathome/1995/06/20': {'image_url': 'https://assets.amuniversal.com/41ab4d9a4c48102dbf94001438c0f03b',
   'likes': 18,
   'favorites': 22,
   'comments': 9,
   'date': 'June_20_1995'},
  'https://www.gocomics.com/adamathome/1995/06/21': {'image_url': 'https://assets.amuniversal.com/4233e2b84c48102dbf94001438c0f03b',
   'likes': 3,
   'favorites': 4,
   'comments': 2,
   'date': 'June_21_1995'},
  'https://www.gocomics.com/adamathome/1995/06/22': {'image_url': 'https://assets.amuniversal.com/401576864c48102dbf94001438c0f03b',
   'likes': 3,
   'favorites': 3,
   'comments': 0,
   'date': 'June_22_1995'},
  'https://www.gocomics.com/adamathome/1995/06/23': {'image_url': 'https://assets.amuniversal.com/410633824c48102dbf94001438c0f03b',
   'likes': 2,
   'favorites': 3,
   'comments': 1,
   'date': 'June_23_1995'},
  'https://www.gocomics.com/adamathome/1995/06/24': {'image_url': 'https://assets.amuniv

In [ ]:
#store category of the comic (save a dict for each comic)
#save data to nas

# data, url, likes, favoriates, comments, categories, 
# +: author (actually no need for that)

In [1]:
not True

False

In [ ]:
## send Giulio a UCB class

- number_arms
- arms means
- alpha

def update(passed_reward)

def get_arm():
    
